<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c5efd4a23e7dbc246ee9a6a9b3e4242384f9c709151c48c3f9bb6a9df811ca3e
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 13:46:04
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.69 L (-1.87%)
Current PnL: -27.02 L (-17.46%)
CY Booked + Current PnL: -12.73 L (-8.22%)
-------------------
Total profit:  1.40 L
Total loss:  -28.42 L
-------------------
Total Booked + Current PnL: 14.25 L (11.15%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.65 L (64.88%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.75,-11.18,12.61,0.02,19550.0,-19522.0,155037.0,136.61,57.0,M-SC,15.45,234.0,-1.00,1.14,34.86,OX40N,NTT,PAINTS
43,ITC,452.00,0.29,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.25,X40,NTT,FMCG
50,MASFIN,398.61,0.02,-1.99,24.53,22.05,23556.0,-1950.0,96030.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.70,39.59,XR,ATH,FINANCE
35,ICICIGI,2252.93,0.33,5.57,13.85,20.19,25489.0,9718.0,184038.0,-17.20,47.0,X-MC,6.61,68.0,0.38,1.35,21.53,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.56,-44.51,213.86,74.15,194142.0,-72825.0,90780.0,-263.44,43.0,H-SC,20.01,149.0,-0.38,0.67,16.37,XR,NTT,CHEMICALS
40,INDUSINDBK,1800.00,3.01,-35.59,106.33,32.89,52875.0,-27480.0,49727.0,-714.28,67.0,L-MC,6.25,259.0,-0.52,0.36,36.94,XR,NTT,BANKS
32,HCLTECH,1922.01,2.79,9.43,13.98,24.73,37010.0,22809.0,264733.0,14.46,71.0,X-LC,5.26,7.0,0.62,1.94,26.10,X40,ATH,IT
36,ICICIPRULI,790.00,2.17,3.53,26.50,30.97,48489.0,6240.0,182978.0,-20.80,56.0,X-MC,2.27,75.0,0.13,1.34,16.54,X40,ATH,INSURANCE
29,HAPPSTMNDS,1480.71,2.17,-36.59,191.51,84.85,155644.0,-46893.0,81272.0,-24.68,53.0,H-SC,14.03,147.0,-0.30,0.60,4.31,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.00,-79.77,516.76,24.79,263336.0,-200897.0,50959.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
44,JCHAC,2282.00,-6.97,-36.16,56.62,-0.01,46197.0,-46213.0,81592.0,16595.65,7.0,M-SC,1.07,233.0,-1.00,0.60,0.00,OX40N,NTT,AC
33,HINDUNILVR,2922.00,-5.35,-9.34,25.38,13.67,59148.0,-24013.0,233050.0,-16.11,31.0,X-LC,3.30,14.0,-0.41,1.71,10.02,XY25,NTT,FMCG
51,MEDANTA,1486.00,-4.34,-3.16,28.13,24.08,34583.0,-4011.0,122939.0,-10.94,33.0,X-SC,6.03,89.0,-0.12,0.90,15.84,XY24,NTT,HEALTHCARE
48,KPIGREEN,730.66,-3.87,-13.68,70.24,46.95,75970.0,-17139.0,108158.0,-20.34,35.0,H-SC,3.03,154.0,-0.23,0.79,35.50,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.78,-6.78,123.04,107.91,169342.0,-10012.0,137632.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.75,-11.18,12.61,0.02,19550.0,-19522.0,155037.0,136.61,57.0,M-SC,15.45,234.0,-1.00,1.14,34.86,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.61,-24.37,50.04,13.47,102052.0,-65727.0,203940.0,-68.90,30.0,H-SC,2.40,158.0,-0.64,1.50,3.70,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.38,-24.39,74.63,32.03,74387.0,-32161.0,99674.0,-18.22,48.0,H-SC,7.86,148.0,-0.43,0.73,11.64,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.21,6.66,61.51,72.27,111170.0,11287.0,180735.0,-8.35,73.0,M-LC,2.95,99.0,0.1,1.33,13.25,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.21,6.66,61.51,72.27,111170.0,11287.0,180735.0,-8.35,73.0,M-LC,2.95,99.0,0.10,1.33,13.25,XR,NTT,IT
38,INDIAMART,4810.62,-1.35,-1.25,109.34,106.72,133170.0,-1542.0,121794.0,-51.01,36.0,H-SC,1.55,139.0,-0.01,0.89,21.84,AR,ATH,MISC
37,IEX,219.00,-2.20,-1.18,50.40,48.63,99587.0,-2361.0,197593.0,-34.34,58.0,H-SC,14.76,136.0,-0.02,1.45,10.24,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.02,-1.42,37.59,35.63,77658.0,-2983.0,206591.0,-15.41,42.0,H-MC,3.41,119.0,-0.04,1.52,15.01,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.78,-6.78,123.04,107.91,169342.0,-10012.0,137632.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.00,-79.77,516.76,24.79,263336.0,-200897.0,50959.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.84,-21.35,88.75,48.45,139961.0,-42804.0,157702.0,-26.96,12.0,X-MC,9.42,64.0,-0.31,1.16,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.17,-37.69,116.98,35.20,93792.0,-48492.0,80178.0,5.13,24.0,X-SC,14.41,92.0,-0.52,0.59,0.13,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.04,-10.91,36.47,21.58,45532.0,-15291.0,124848.0,-55.08,27.0,X-MC,5.72,56.0,-0.34,0.92,10.53,X40N,ATH,IT
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.39,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.86,-24.11,116.29,64.15,210008.0,-57361.0,180590.0,-55.44,39.0,X-SC,1.52,82.0,-0.27,1.32,1.30,XY24,BTT,CEMENT
43,ITC,452.0,0.29,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.25,X40,NTT,FMCG
8,AWL,485.0,1.47,-20.50,93.07,53.50,223271.0,-61855.0,239896.0,-61.46,34.0,X-MC,1.58,58.0,-0.28,1.76,6.55,XY24,NTT,FMCG
15,CAMS,4762.0,-80.00,-79.77,516.76,24.79,263336.0,-200897.0,50959.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
36,ICICIPRULI,790.0,2.17,3.53,26.50,30.97,48489.0,6240.0,182978.0,-20.80,56.0,X-MC,2.27,75.0,0.13,1.34,16.54,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.00,-79.77,516.76,24.79,263336.0,-200897.0,50959.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-0.84,-21.35,88.75,48.45,139961.0,-42804.0,157702.0,-26.96,12.0,X-MC,9.42,64.0,-0.31,1.16,0.00,X40N,ATH,FINANCE
54,PGHH,17907.65,-0.31,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,28.0,X-MC,4.68,57.0,-0.15,1.39,0.00,X40,ATH,FMCG
55,QUESS,424.00,-2.25,-31.69,109.10,42.84,48448.0,-20599.0,44407.0,-55.37,34.0,X-SC,37.35,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-1.17,-37.69,116.98,35.20,93792.0,-48492.0,80178.0,5.13,24.0,X-SC,14.41,92.0,-0.52,0.59,0.13,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.0,-2.25,-31.69,109.10,42.84,48448.0,-20599.0,44407.0,-55.37,34.0,X-SC,37.35,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.0,-80.00,-79.77,516.76,24.79,263336.0,-200897.0,50959.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
59,RELAXO,1176.0,-0.84,-47.84,191.99,52.31,145369.0,-69443.0,75717.0,-44.44,33.0,X-SC,5.40,91.0,-0.48,0.56,1.35,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-1.17,-37.69,116.98,35.20,93792.0,-48492.0,80178.0,5.13,24.0,X-SC,14.41,92.0,-0.52,0.59,0.13,X40,NTT,FOOTWEAR
51,MEDANTA,1486.0,-4.34,-3.16,28.13,24.08,34583.0,-4011.0,122939.0,-10.94,33.0,X-SC,6.03,89.0,-0.12,0.90,15.84,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.86,-11.99,35.53,19.28,108177.0,-41490.0,304466.0,-23.30,70.0,X-LC,6.53,1.0,-0.38,2.23,12.56,X40,ATH,IT
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.50,17.77,X40,BTT,IT
75,TMPV,600.00,-0.11,-15.53,68.33,42.18,159533.0,-42935.0,233475.0,-24.30,32.0,X-LC,3.55,3.0,-0.27,1.71,1.13,XY24,NTT,AUTO
81,VBL,671.64,0.37,-3.19,40.14,35.67,122733.0,-10080.0,305762.0,-14.67,59.0,X-LC,5.31,4.0,-0.08,2.24,10.22,X40N,ATH,FMCG
43,ITC,452.00,0.29,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.45,4.25,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,3.01,-35.59,106.33,32.89,52875.0,-27480.0,49727.0,-714.28,67.0,L-MC,6.25,259.0,-0.52,0.36,36.94,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.39,-17.11,119.16,81.67,92661.0,-16048.0,77762.0,6988.89,38.0,L-SC,16.28,271.0,-0.17,0.57,50.59,XR,NTT,CERAMICS
50,MASFIN,398.61,0.02,-1.99,24.53,22.05,23556.0,-1950.0,96030.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.70,39.59,XR,ATH,FINANCE
13,BSOFT,831.70,-0.39,-21.28,91.55,50.79,101365.0,-29932.0,110721.0,0.89,74.0,H-SC,5.20,151.0,-0.30,0.81,28.92,XR,ATH,IT
39,INDIGOPNTS,1408.00,-2.75,-11.18,12.61,0.02,19550.0,-19522.0,155037.0,136.61,57.0,M-SC,15.45,234.0,-1.00,1.14,34.86,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.75,-11.18,12.61,0.02,19550.0,-19522.0,155037.0,136.61,57.0,M-SC,15.45,234.0,-1.00,1.14,34.86,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.20,3.57,16.77,20.95,43730.0,8994.0,260762.0,2.46,80.0,X-LC,18.47,31.0,0.21,1.91,40.48,X40,ATH,PAINTS
13,BSOFT,831.70,-0.39,-21.28,91.55,50.79,101365.0,-29932.0,110721.0,0.89,74.0,H-SC,5.20,151.0,-0.30,0.81,28.92,XR,ATH,IT
40,INDUSINDBK,1800.00,3.01,-35.59,106.33,32.89,52875.0,-27480.0,49727.0,-714.28,67.0,L-MC,6.25,259.0,-0.52,0.36,36.94,XR,NTT,BANKS
41,INFY,2275.00,1.20,8.76,42.40,54.87,144283.0,27401.0,340289.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.50,17.77,X40,BTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.67
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.07
MC    30.18
LC    24.75
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.34
X40      22.41
X40N     10.86
XR       10.05
AR        9.43
XY25      9.33
OX40N     7.76
SR        1.03
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.58
X-MC    23.44
X-LC    21.17
M-SC    12.16
X-SC     6.93
H-MC     4.96
M-MC     1.42
L-SC     1.40
M-LC     1.33
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.35,-6.37,40.77
IT,12.81,-15.44,77.85
FINANCE,9.60,-18.32,68.32
ELECTRICAL,6.26,-10.13,50.48
PAINTS,6.20,-9.37,25.61
MISC,6.16,-54.85,118.29
INSURANCE,4.52,-2.51,38.64
PHARMA,4.09,-0.45,32.59
AUTO,3.46,-21.93,74.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3173640.0,21
XR,1290158.0,13
AR,1289261.0,10
X40,983471.0,14
X40N,967925.0,9
OX40N,735528.0,10
XY25,368591.0,6
SR,280650.0,2
MH,75970.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3627449.0,25
M-SC,1402423.0,15
X-MC,1289288.0,16
X-SC,975183.0,8
X-LC,874711.0,11
H-MC,404146.0,3
L-SC,270081.0,3
M-LC,111170.0,1
H-LC,67929.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1273598.0      6
           AR         886194.0      5
M-SC       XY24       797512.0      6
H-SC       XR         773267.0      7
X-SC       X40N       503535.0      3
X-MC       X40        448280.0      7
           XY24       384289.0      3
X-LC       X40        356582.0      5
H-SC       OX40N      337770.0      4
M-SC       OX40N      309621.0      5
X-SC       XY24       293039.0      3
H-SC       SR         280650.0      2
X-MC       X40N       269042.0      4
X-LC       XY24       231806.0      2
H-MC       AR         210750.0      2
X-LC       X40N       195348.0      2
H-MC       XY24       193396.0      1
X-MC       XY25       187677.0      2
L-SC       XR         181944.0      2
X-SC       X40        178609.0      2
M-SC       XR         170902.0      2
           AR         124388.0      2
M-LC       XR         111170.0      1
X-LC       XY25        90975.0      2
L-SC       OX40N       88137.0      1
H-SC       MH          75970.0      1
H-LC       AR          67929.0      1
L-MC       XR          52875.0      1
M-MC       XY25        51171.0      1
L-LC       XY25        38768.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
